In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [7]:
STORM_NUMBER = 10
TITLE = "1 January 2021"
SUBTITLE = "8 AM 2 January 2021"
sts = utc(2021, 1, 1, 18)
ets = utc(2021, 12, 30, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 40, 'z': 'yes', 'f': 'linear'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [8]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [11]:
cull = []
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
    ['IA-LN-31', 0.5],
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=4)
res = mp.postprocess(filename='210102.png')
mp.close()

     state  wfo  val       lon      lat                              geo  \
426     WI  MKX  3.9  -87.9800  43.2400   POINT (969402.322 -124083.453)   
427     NH  GYX  3.8  -71.8100  43.8200   POINT (2205248.913 258296.996)   
428     WI  MKX  3.8  -88.1100  43.2300   POINT (959170.447 -126726.078)   
429     VT  BTV  3.8  -73.2500  44.3000   POINT (2080708.832 270919.024)   
430     MI  DTX  3.8  -83.9700  43.1200   POINT (1290815.520 -81572.462)   
...    ...  ...  ...       ...      ...                              ...   
3050    ND  FGF  0.0  -97.1667  46.0667    POINT (218540.224 122449.245)   
3051    ND  FGF  0.0  -97.2400  46.4000    POINT (211606.637 159295.554)   
3052    ND  FGF  0.0  -97.2386  48.9742    POINT (201619.976 445317.908)   
3053    NE  CYS  0.0 -103.7347  42.4244  POINT (-306491.436 -279383.054)   
3054    NE  CYS  0.0 -103.3678  41.5639  POINT (-280222.293 -376281.684)   

      used_for_analysis  nwsli  plotme source  
426                True    426    True 